In [2]:
import yaml
import os
# !pip install e3nn
from e3nn import o3

In [3]:
def load_config(config_file_path):
    with open(config_file_path, 'r') as stream:
        try:
            config = yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)
    return config

def save_config(config, config_file_path):
    with open(config_file_path, 'w') as stream:
        try:
            yaml.dump(config, stream, default_flow_style=False)
        except yaml.YAMLError as exc:
            print(exc)

def update_config(config, **kwargs):
    for key, value in kwargs.items():
        config[key] = value
    return config

In [5]:
example_config_file_path="old_configs/water.yaml"
num_features=32
max_epochs=200
root="results/toluene"

'''
feature_irreps_hidden: 32x0o + 32x0e + 32x1o + 32x1e                              # irreps used for hidden features, here we go up to lmax=1, with even and odd parities; for more accurate but slower networks, use l=2 or higher, smaller number of features is faster
irreps_edge_sh: 0e + 1o                                                           # irreps of the spherical harmonics used for edges. If a single integer, indicates the full SH up to L_max=that_integer
conv_to_output_hidden_irreps_out: 16x0e                                           # irreps used in hidden layer of output block
max_epochs: 150                                                                  # stop training after _ number of epochs, we set a very large number here, it won't take this long in practice and we will use early stopping instead

num_layers: 3                                                                     # number of interaction blocks, we find 3-5 to work best
invariant_layers: 1
max_epochs: 150                                                                  # stop training after _ number of epochs, we set a very large number here, it won't take this long in practice and we will use early stopping instead
'''

for n_train in [10,100,1000]:
    for conv0_layers in range(3):
        for lmax in range(4):

            config = load_config(example_config_file_path)

            num_layers=4-conv0_layers
            feature_irreps_hidden=str(o3.Irreps(
                            [
                                (num_features, (l, p))
                                for p in ((1, -1) if True else (1,))
                                for l in range(lmax + 1)
                            ]
                        ))
            run_name=f"ntrain_{n_train}_lmax_{lmax}_invariant_{conv0_layers}_epochs_{max_epochs}"

            config = update_config(config, num_layers=num_layers, feature_irreps_hidden=feature_irreps_hidden, max_epochs=max_epochs, root=root, run_name=run_name, conv0_layers=conv0_layers, n_train=n_train)
            save_config(config, f"water_configs/{run_name}.yaml")